In [ ]:
import sys
import shap
import matplotlib.pyplot as plt

from itertools import compress
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import make_scorer
from audioop import cross
from tabnanny import verbose
from sklearn.model_selection import cross_validate
from import_library import *
from model_stack import model_list
from func import loaders
from xgboost import plot_importance

sys.path.append('..')

In [ ]:
print("CPU count: {}".format(multiprocessing.cpu_count()))
data_type ='mean_var_cv' #mean, var, mix,First_mix,mean_var_cv
data_value_type = 'ALFF_mvc' #ALFF, FC,ALFF_mix,FC_mix, ALFF_mvc, FC_mvc
grid = True
kfold = KFold(n_splits=5)
# kfold = KFold(n_splits=5,shuffle=True)
cross_v = True

In [ ]:
# Data from R-boruta feature selection

if data_value_type == 'ALFF_mix' or data_value_type == 'FC_mix':
    feature_selection_df_savepath='./feature_selection/feature_selection_result_R_boruta(mix)/'
elif data_value_type == 'FC_mvc':
    feature_selection_df_savepath = './feature_selection/feature_selection_result_R_boruta(m+v+c)/'
elif data_value_type == 'ALFF_mvc':
    feature_selection_df_savepath = './feature_selection/feature_selection_result_R_boruta(m+v+c)/'
else:
    feature_selection_df_savepath = './feature_selection/feature_selection_result_R_boruta/'

# dataloader
clinical_data = pd.read_csv('./data/clinical data/clinical_data_2.csv')[['subj', 'AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]']]
raw_data_path = './For_Regression_dataset/'
next_path = [i for i in os.listdir('./For_Regression_dataset/') if 'MOCA'in i or 'AD8' in i]

# feature selection process
rank_threshold = 'R-Boruta-Results'

def feature_filter(patient_value, load_path = None, outcome_name = None, data_type_ = 'mean', data_value_type = None):
    feature_rank_df = pd.read_csv(load_path, index_col = False).rename(columns={'Unnamed: 0': 'feature_name'})
    feature_rank_df = feature_rank_df[['feature_name', 'decision']]

    if rank_threshold == 'R-Boruta-Results':
        feature_rank_df = feature_rank_df[feature_rank_df['decision']!='Rejected']
    else:
        feature_rank_df = feature_rank_df.head(rank_threshold)

    patient_value = patient_value[feature_rank_df['feature_name'].tolist()]

    return feature_rank_df['feature_name'].tolist(), patient_value

def loader_(data_path = None ,mat_dtype='FCM_mean', outcome_name = None):
    outcome_name_type = ("AD8" if "AD8" in outcome_name else "MOCA")

    if data_type =='mix':
        if data_value_type =='FC': 
            
            # MOCA客觀認知測驗分數[<=23,>=24]_mean_44850_FC.csv
            pd_tb_1 = pd.read_csv(os.path.join(data_path[0], '{}_mean_{}_{}.csv'.format(outcome_name, 44850, data_value_type))) 
            pd_tb_2 = pd.read_csv(os.path.join(data_path[0], '{}_var_{}_{}.csv'.format(outcome_name, 44850, data_value_type)))
        else:
            pd_tb_1, pd_tb_2 = pd.read_csv(os.path.join(data_path[0], 'mean_mdALFF.csv')), pd.read_csv(os.path.join(data_path[0], 'var_mdALFF.csv'))

        pd_patient_index_1, pd_patient_index_2 = pd_tb_1[pd_tb_1.columns[0]]+'_m', pd_tb_2[pd_tb_2.columns[0]]+'_v'
        pd_patient_value_1, pd_patient_value_2 = pd_tb_1[pd_tb_1.columns[1::]], pd_tb_2[pd_tb_2.columns[1::]]

        fd_csv_name = ("R-Boruta-{}-{}_{}_feature-selection_result.csv".format(data_value_type, outcome_name_type, 'mean'))
        col_1, pd_patient_value_1 = feature_filter(pd_patient_value_1, load_path = os.path.join(feature_selection_df_savepath, fd_csv_name), outcome_name = outcome_name, data_type_ = 'mean', )
        
        fd_csv_name = ("R-Boruta-{}-{}_{}_feature-selection_result.csv".format(data_value_type, outcome_name_type, 'var'))
        col_2, pd_patient_value_2 = feature_filter(pd_patient_value_2, load_path = os.path.join(feature_selection_df_savepath, fd_csv_name), outcome_name = outcome_name, data_type_ = 'var', )

        pd_patient_value_mix = pd.DataFrame(np.concatenate((pd_patient_value_1, pd_patient_value_2), axis=1), columns=col_1+col_2)

        return col_1+col_2, pd_patient_value_mix.values
        
    elif data_type == 'mean_var_cv':
        if data_value_type == 'ALFF_mvc':
            overall_feature_df = 'ALFF(mean+var+cv)dataset.csv'
            pd_tb = pd.read_csv(os.path.join(data_path[0], overall_feature_df))
            pd_patient_index = pd_tb[pd_tb.columns[0]]
            pd_patient_value = pd_tb[pd_tb.columns[1::]]

            fd_csv_name = ("R-Boruta-{}-{}_feature-selection-result.csv".format(data_value_type,outcome_name_type))
            pd_patient_index, pd_patient_value = feature_filter(pd_patient_value, load_path=os.path.join(feature_selection_df_savepath, fd_csv_name), outcome_name = outcome_name_type)

            return pd_patient_index,pd_patient_value.values
        else:
                overall_feature_df = 'FC(mean+var+cv)dataset.csv'
                pd_tb = pd.read_csv(os.path.join(data_path[0], overall_feature_df))
                pd_patient_index = pd_tb[pd_tb.columns[0]]
                pd_patient_value = pd_tb[pd_tb.columns[1::]]

                fd_csv_name = ("R-Boruta-{}-{}_feature-selection-result.csv".format(data_value_type,outcome_name_type))
                pd_patient_index, pd_patient_value = feature_filter(pd_patient_value, load_path=os.path.join(feature_selection_df_savepath, fd_csv_name), outcome_name = outcome_name_type)

                return pd_patient_index,pd_patient_value.values
    else:
        if data_type =='First_mix':
            if data_value_type == 'ALFF_mix':
                overall_feature_df = 'ALFF_mix(check).csv'

                pd_tb = pd.read_csv(os.path.join(data_path[0], overall_feature_df))
                pd_patient_index = pd_tb[pd_tb.columns[0]]
                pd_patient_value = pd_tb[pd_tb.columns[1::]]

                fd_csv_name = ("R-Boruta-{}-{}_feature-selection-result.csv".format(data_value_type,outcome_name_type))
                pd_patient_index, pd_patient_value = feature_filter(pd_patient_value, load_path=os.path.join(feature_selection_df_savepath, fd_csv_name), outcome_name = outcome_name_type)

                return pd_patient_index,pd_patient_value.values
            else:
                overall_feature_df = 'MOCA客觀認知測驗分數[<=23,>=24]_mix_FC.csv'

                pd_tb = pd.read_csv(os.path.join(data_path[0], overall_feature_df))
                pd_patient_index = pd_tb[pd_tb.columns[0]]
                pd_patient_value = pd_tb[pd_tb.columns[1::]]

                fd_csv_name = ("R-Boruta-{}-{}_feature-selection-result.csv".format(data_value_type,outcome_name_type))
                pd_patient_index, pd_patient_value = feature_filter(pd_patient_value, load_path=os.path.join(feature_selection_df_savepath, fd_csv_name), outcome_name = outcome_name_type)

                return pd_patient_index,pd_patient_value.values


        else:
            if data_type == 'mean':
                if data_value_type =='FC': 

                    overall_feature_df = '{}_{}_{}_{}.csv'.format(outcome_name, data_type, 44850, data_value_type)

                    pd_tb = pd.read_csv(os.path.join(data_path[0], overall_feature_df))

                    pd_patient_index = pd_tb[pd_tb.columns[0]]
                    pd_patient_value = pd_tb[pd_tb.columns[1::]]

                    fd_csv_name = ("R-Boruta-{}-{}_{}_feature-selection_result.csv".format(data_value_type, outcome_name_type, data_type))
                    pd_patient_index, pd_patient_value = feature_filter(pd_patient_value, load_path=os.path.join(feature_selection_df_savepath, fd_csv_name), outcome_name = outcome_name_type)
                    
                    return pd_patient_index, pd_patient_value.values

                if data_value_type =='ALFF':
                    overall_feature_df = '{}_mdALFF.csv'.format(data_type)

                    pd_tb = pd.read_csv(os.path.join(data_path[0], overall_feature_df))

                    pd_patient_index = pd_tb[pd_tb.columns[0]]
                    pd_patient_value = pd_tb[pd_tb.columns[1::]]

                    fd_csv_name = ("R-Boruta-{}-{}_{}_feature-selection_result.csv".format(data_value_type, outcome_name_type, data_type))
                    pd_patient_index, pd_patient_value = feature_filter(pd_patient_value, load_path=os.path.join(feature_selection_df_savepath, fd_csv_name), outcome_name = outcome_name_type)

                    return pd_patient_index, pd_patient_value.values
            else:
                if data_value_type =='FC': 
                    overall_feature_df = '{}_{}_{}_{}.csv'.format(outcome_name, data_type, 44850, data_value_type)

                    pd_tb = pd.read_csv(os.path.join(data_path[0], overall_feature_df))

                    pd_patient_index = pd_tb[pd_tb.columns[0]]
                    pd_patient_value = pd_tb[pd_tb.columns[1::]]

                    fd_csv_name = ("R-Boruta-{}-{}_{}_feature-selection_result.csv".format(data_value_type, outcome_name_type, data_type))
                    pd_patient_index, pd_patient_value = feature_filter(pd_patient_value, load_path=os.path.join(feature_selection_df_savepath, fd_csv_name), outcome_name = outcome_name_type)

                    return pd_patient_index, pd_patient_value.values
                if data_value_type =='ALFF':
                    overall_feature_df = '{}_mdALFF.csv'.format(data_type)

                    pd_tb = pd.read_csv(os.path.join(data_path[0], overall_feature_df))

                    pd_patient_index = pd_tb[pd_tb.columns[0]]
                    pd_patient_value = pd_tb[pd_tb.columns[1::]]
                    
                    fd_csv_name = ("R-Boruta-{}-{}_{}_feature-selection_result.csv".format(data_value_type, outcome_name_type, data_type))
                    pd_patient_index, pd_patient_value = feature_filter(pd_patient_value, load_path=os.path.join(feature_selection_df_savepath, fd_csv_name), outcome_name = outcome_name_type)

                    return pd_patient_index, pd_patient_value.values

In [ ]:
result_dict={'outcome':[], 'model':[], 'Reg:mae':[], 'Reg:mse':[],'Reg:rmse':[]}
grid_stack, scores_stack = [], []

def fxn():
    warnings.warn("deprecated", DeprecationWarning)
if True:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        fxn()

        model_stack = model_list(grid=grid)
        
        
        for outcome_name in ["AD8主觀認知障礙[0-1,>=2]", "MOCA客觀認知測驗分數[<=23,>=24]"]:
            for model in model_stack:
                if model in ['CATBOOST','MLPR', 'XBGR', 'LGBMR']:

                    result_dict['outcome'].append(outcome_name)
                    result_dict['model'].append(model)

                    label_ = clinical_data[outcome_name]

                    if 'AD8' in outcome_name:
                        label_ = label_+1
                    _, patient_value = loader_(data_path = [raw_data_path, next_path], outcome_name = outcome_name)

                    print("Overall: ",patient_value.shape)
                    print("Feature Number: {}".format(patient_value.shape[1]))
                    if grid==False:
                        print()
                        if data_value_type =='FC':
                            print("FC Outcome: {}[{}] - cut rank point: {} - GridSearch: {}".format(outcome_name, data_type, rank_threshold, grid))
                        else:
                            print("ALFF Outcome: {}[{}] - cut rank point: {} - GridSearch: {}".format(outcome_name, data_type, rank_threshold, grid))
                        print("--Start Training {} model--".format(model))
                        if cross_v==True:
                            scoring = {'mae': make_scorer(mean_absolute_error), 'mse': make_scorer(mean_squared_error), 'mape':make_scorer(mean_absolute_percentage_error)}
                            scores = cross_validate(model_stack[model], patient_value, np.array(label_), scoring=scoring, cv = 5,  return_train_score=True)
                            scores_stack.append(scores)
                        else:
                            model = model_stack[model].fit(patient_value, np.array(label_))
                            overall_pred =  model.predict(patient_value)
                        if cross_v==True:
                            result_dict['Reg:mae'].append("{} - {} ({})".format(round(np.min(scores['test_mae']),5), round(np.max(scores['test_mae']),5), round(np.mean(scores['test_mae']),5)))
                            result_dict['Reg:mse'].append("{} - {} ({})".format(round(np.min(scores['test_mse']),5), round(np.max(scores['test_mse']),5), round(np.mean(scores['test_mse']),5)))
                            result_dict['Reg:mape'].append("{} - {} ({})".format(round(np.min(scores['test_mape']),5), round(np.max(scores['test_mape']),5), round(np.mean(scores['test_mape']),5))) 
                        else:
                            result_dict['Reg:mae'].append(round(mean_absolute_error(np.array(label_),overall_pred), 5))
                            result_dict['Reg:mse'].append(round(mean_squared_error(np.array(label_),overall_pred),5))
                            result_dict['Reg:mape'].append(round(mean_absolute_percentage_error(np.array(label_),overall_pred),5))
                    else:
                        if data_value_type =='FC':
                            print("FC Outcome: {}[{}] - cut rank point: {} - GridSearch: {}".format(outcome_name, data_type, rank_threshold, grid))
                        elif  data_value_type =='FC_mix':
                            print("======================================FC_Mix======================================")
                            print(outcome_name)
                        elif data_value_type =='ALFF_mix':
                            print("=====================================ALFF_Mix=====================================")
                            print(outcome_name)
                        elif data_value_type == 'ALFF_mvc':
                            print("=====================================ALFF_(m+v+c)=================================")
                            print(outcome_name)
                        elif data_value_type == 'FC_mvc':
                            print("=====================================FC_(m+v+c)===================================")
                            print(outcome_name)
                        else:
                            print("ALFF Outcome: {}[{}] - cut rank point: {} - GridSearch: {}".format(outcome_name, data_type, rank_threshold, grid))
                        print("--Start Training {} model--".format(model))
                        
                        scoring = {'mae': make_scorer(mean_absolute_error), 'mse': make_scorer(mean_squared_error),
                                                    'rmse':make_scorer(mean_squared_error,squared=False)}

                        GridModel = GridSearchCV(model_stack[model][0],model_stack[model][1], cv=kfold,  n_jobs = 20, refit = 'neg_mean_absolute_percentage_error', scoring="neg_mean_absolute_percentage_error",
                                                    return_train_score=True)

                        print(patient_value.shape, np.array(label_).shape)

                        GridModel.fit(patient_value, np.array(label_))

                        print('Best parameter : [{}]\n'.format(GridModel.best_params_))

                        param_ = GridModel.best_params_
                        model_stack[model][0].set_params(**param_)

                        scores = cross_validate(model_stack[model][0], patient_value, np.array(label_),  scoring=scoring, cv = 5,  n_jobs = -1, return_train_score=True)
                        scores_stack.append(scores)

                        result_dict['Reg:mae'].append("{}".format( round(np.mean(scores['test_mae']),5)))
                        result_dict['Reg:mse'].append("{} ".format(round(np.mean(scores['test_mse']),5)))
                        result_dict['Reg:rmse'].append("{}".format( round(np.mean(scores['test_rmse']),5))) 
                        
                        # print("pass",model)
                        grid_stack.append(GridModel)
                    # print('----------------------------------------------------')

In [ ]:
#ALFF
for i in grid_stack:
    print(i.best_params_)
    print('----------------------------------------------------------------------------------------------------------------------------------------')

In [ ]:
#FC
for i in grid_stack:
    print(i.best_params_)
    print('----------------------------------------------------------------------------------------------------------------------------------------')

In [ ]:
#ALFF
result_df = pd.DataFrame(result_dict) 
result_df.style.set_caption("AD8/MOCA - [{} - {} - Grid: {} - CV: {}]".format(data_value_type ,data_type, grid, cross_v))
result_df.to_csv("./metrics_Result/AD8-MOCA - [{} - {} - Grid: {} - CV: {}] RT = top {}.csv".format(data_value_type ,data_type, grid, cross_v, rank_threshold))
result_df

In [ ]:
#FC(No Standardization values)
result_df = pd.DataFrame(result_dict) 
result_df.style.set_caption("AD8/MOCA - [{} - {} - Grid: {} - CV: {}]".format(data_value_type ,data_type, grid, cross_v))
result_df.to_csv("./metrics_Result/AD8-MOCA - [{} - {} - Grid: {} - CV: {}] RT = top {}.csv".format(data_value_type ,data_type, grid, cross_v, rank_threshold))
result_df

In [ ]:
#FC(Standardization values)
result_df = pd.DataFrame(result_dict) 
result_df.style.set_caption("AD8/MOCA - [{} - {} - Grid: {} - CV: {}]".format(data_value_type ,data_type, grid, cross_v))
# result_df.to_csv("./metrics_Result/AD8-MOCA - [{} - {} - Grid: {} - CV: {}] RT = top {}.csv".format(data_value_type ,data_type, grid, cross_v, rank_threshold))
result_df